# **Build Knowledge Graph RAG With LlamaIndex**

![image.png](https://images.pexels.com/photos/30381207/pexels-photo-30381207/free-photo-of-modern-trading-desk-with-graphs-and-gadgets.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=2)

## **Approach**


* **Data Collection:** Gather financial data from multiple sources such as stock feeds, news websites, and industry reports using Google search engine API and scraping techniques


* **Knowledge Graph Construction:** Build a knowledge graph to represent relationships between extracted entities like companies, market trends, and sectors using neo4j


* **Vector Embedding and Indexing**
  - Convert textual data into vector embeddings using OpenAI
  - Index data using LlamaIndex to enable retrieval-augmented generation

* RAG Workflow Integration
  - Query the vector database to fetch relevant context and related entities from the knowledge graph
  - Use OpenAI to generate contextually accurate and actionable responses


In [ ]:
!pip install llama-index
!pip install llama-index-llms-openai
!pip install llama-index-graph-stores-neo4j
!pip install llama-index-embeddings-openai

In [11]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
import os
import openai
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
import nest_asyncio
import asyncio

In [5]:
google_api = "enterapi_key"
search_engine_id = "enter_id"
open_ai_key = "enter_openai_key"

In [12]:
# Apply nest_asyncio
nest_asyncio.apply()

In [9]:
os.environ["OPENAI_API_KEY"] = open_ai_key
openai.api_key = os.environ["OPENAI_API_KEY"]

In [6]:
import requests

def search_with_google_api(query):
    url = f"https://www.googleapis.com/customsearch/v1?q={query}&key={google_api}&cx={search_engine_id}"

    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get("items", [])
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []


In [18]:
import os
import requests
from bs4 import BeautifulSoup
import openai


def generate_search_queries(user_input):
    """
    Generates a list of 5-7 detailed and relevant search queries for financial sentiment analysis
    based on the user's input, such as a target sector, field, or region.
    """
    prompt = f"""
    You are a financial analyst and search query expert. Based on the following user input, generate a list of 5-7 search queries
    for financial sentiment analysis based on user input. Ensure the queries cover diverse aspects of the topic, including sector-specific trends,
    regional financial overviews, and broader financial landscapes. The queries should focus on extracting data relevant to sentiment
    and performance analysis.

    User Input: {user_input}

    Strictly output the queries as a python list of strings. Do not add any additional comments.
    """

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert in generating search queries for financial sentiment analysis."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=200
    )

    # Extract and clean up the list of queries
    queries =  response.choices[0].message.content.strip()
    return eval(queries)

def fetch_full_content(url):
    """
    Fetches the full content of a webpage given its URL.
    """
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
        )
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            paragraphs = soup.find_all("p")
            full_text = "\n".join([p.get_text() for p in paragraphs])
            return full_text.strip() if full_text else None
        else:
            print(f"Error: Unable to fetch content from {url} (Status Code: {response.status_code})")
            return None
    except Exception as e:
        print(f"Error fetching content from {url}: {e}")
        return None

def create_dataset_from_queries(queries, directory="dataset"):
    """
    Process search queries and save results as text files in the same directory.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)

    file_count = 1  # To ensure unique filenames across all queries

    for query in queries:
        print(f"Processing query: {query}")
        valid_count = 0
        page_number = 1

        while valid_count < 10:
            print(f"Fetching search results, page {page_number}...")
            results = search_with_google_api(query + f"&start={page_number * 10}")

            if not results:
                print("No more results found. Try refining the query.")
                break

            for result in results:
                if valid_count >= 10:
                    break  # Stop when 10 valid documents are saved

                title = result["title"]
                link = result["link"]
                snippet = result.get("snippet", "No snippet")

                # Fetch full content of the link
                full_content = fetch_full_content(link)
                if full_content:  # Save only if content is valid
                    filename = f"{directory}/doc_{file_count}.txt"
                    with open(filename, "w", encoding="utf-8") as f:
                        f.write(f"Query: {query}\n")
                        f.write(f"Title: {title}\n")
                        f.write(f"Link: {link}\n")
                        f.write(f"Snippet: {snippet}\n\n")
                        f.write(f"Full Content:\n{full_content}")
                    print(f"Saved: {filename}")
                    valid_count += 1
                    file_count += 1
                else:
                    print(f"Skipped: {link} (No valid content)")

            page_number += 1  # Move to the next page of results

    print(f"Finished processing all queries. Total files saved: {file_count - 1}")


user_input = "Financial sentiment analysis for the electric vehicle sector in the US"
queries = generate_search_queries(user_input)
queries
create_dataset_from_queries(queries)


Processing query: US electric vehicle sector sentiment analysis
Fetching search results, page 1...
Saved: dataset/doc_1.txt
Error: Unable to fetch content from https://www.spglobal.com/mobility/en/research-analysis/us-ev-sales-grew-nearly-52-in-2023.html (Status Code: 403)
Skipped: https://www.spglobal.com/mobility/en/research-analysis/us-ev-sales-grew-nearly-52-in-2023.html (No valid content)
Saved: dataset/doc_2.txt
Error: Unable to fetch content from https://www.weforum.org/stories/2024/06/china-electric-vehicle-advantage/ (Status Code: 403)
Skipped: https://www.weforum.org/stories/2024/06/china-electric-vehicle-advantage/ (No valid content)
Saved: dataset/doc_3.txt
Error: Unable to fetch content from https://www.spglobal.com/mobility/en/research-analysis/2024-ev-forecast-the-supply-chain-charging-network-and-battery.html (Status Code: 403)
Skipped: https://www.spglobal.com/mobility/en/research-analysis/2024-ev-forecast-the-supply-chain-charging-network-and-battery.html (No valid co

The code `documents = SimpleDirectoryReader("dataset").load_data()` uses a utility class, typically from libraries like llama-index (formerly gpt-index), to read and preprocess all documents stored in a specified directory. Here, the "dataset" directory is scanned for files, usually in formats like .txt. The SimpleDirectoryReader handles tasks such as opening the files, reading their content, and managing encodings, while ignoring unsupported or corrupt files.

In [20]:
documents = SimpleDirectoryReader("dataset").load_data()

Initiate graph store using Neo4j Credentials. Steps are given in the beginning of the notebook.

In [21]:
graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="",
    url="neo4j+s://.databases.neo4j.io",
)

In [22]:
# Create the index
index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    kg_extractors=[
        SchemaLLMPathExtractor(
            llm=OpenAI(model="gpt-3.5-turbo", temperature=0.0)
        )
    ],
    property_graph_store=graph_store,
    show_progress=True,
    use_async=True
)


Parsing nodes:   0%|          | 0/70 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 135/135 [00:07<00:00, 17.22it/s]


In [32]:

# save and load using locally stored graphs if you dont want to implement neo4j, you can directly use locally stored index in storage folder
index.storage_context.persist(persist_dir="./storage")

from llama_index.core import StorageContext, load_index_from_storage

index = load_index_from_storage(
    StorageContext.from_defaults(persist_dir="./storage")
)


## **Building GraphRAG Pipeline**



In [8]:
# loading from existing graph store (and optional vector store)
# load from existing graph/vector store
index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store
)

In [25]:
# Define retriever
retriever = index.as_retriever(
    include_text=False,  # Default is true
)
results = retriever.retrieve("What is the summary of the finanacial texts?")
for record in results:
    print(record.text)

Greeven -> WORKED_ON -> The Economist
Wei -> WORKED_ON -> The Economist
Xue -> WORKED_ON -> The Economist
He -> WORKED_ON -> The Economist
James Hamilton -> WORKED_ON -> Oil and the Macroeconomy since World War II
investor sentiment -> LOCATED_IN -> US
CFO -> WORKED_ON -> gross profit
CFO -> WORKED_ON -> ASP
CFO -> WORKED_ON -> manufacturing credits
CFO -> WORKED_ON -> battery costs
CFO -> WORKED_ON -> IRA incentives


In [28]:
query_engine = index.as_query_engine(include_text=True)
response = query_engine.query("How to invest in the EV sector? Summarize the most important financial trends in the EV Sector.")
print(response)


Investing in the EV sector involves understanding key financial trends. The sector has seen significant growth, with sales of battery electric and plug-in hybrid electric vehicles surpassing two million vehicles in 2019. Despite disruptions like the COVID-19 pandemic, the market continues to expand. Market segmentation is crucial for identifying opportunities and managing risks. Government interventions, consumer attitudes, and OEM investments play a significant role in shaping the sector. The EV market is expected to grow substantially by 2030, presenting opportunities for traditional and new-entrant OEMs, finance companies, and dealerships. Understanding global sales trends, market shares, and production projections, especially in regions like China, can provide valuable insights for potential investors in the EV sector.


In [30]:
query_engine = index.as_query_engine(include_text=True)
response = query_engine.query("Which companies are doing the best in EV sector?")
print(response)


BYD and other Chinese EV car companies are leading the electric vehicle sector.


In [31]:
query_engine = index.as_query_engine(include_text=True)
response = query_engine.query("How is Tesla doing in EV sector?")
print(response)

Tesla is located in various countries such as the United States, China, and India. The company experienced a decline in sales in 2024, marking its first annual sales drop in a decade. Despite this decline, analysts remain confident in Tesla's ability to accelerate delivery growth in the future. Tesla's financials improved in Q2 2024, with revenue increasing by 20% year-over-year driven by a rise in deliveries. The company delivered a significant number of vehicles during the quarter, although overall demand for electric vehicles in the U.S. and elsewhere slowed. Tesla's automotive sector revenue increased over 14% year-over-year in Q2 2024, with revenue from regulatory credits doubling from the previous year. Additionally, income from other ventures like energy storage deployment and charging networks rose significantly.


In [14]:
from typing import List, Dict

def generate_summary_report(context: str, query: str) -> str:
    """
    Generate a detailed summary report for financial sentiment analysis.
    Takes context and query as inputs and returns a comprehensive summary.
    """
    prompt = f"""
    You are a financial sentiment analysis assistant. Using the context provided below, generate a detailed summary report:

    Context:
    {context}

    Query:
    {query}

    The report should include:
    1. A high-level summary of the financial trends related to the query.
    2. Key positive, negative, and neutral sentiments detected.
    3. Reasons or factors driving the sentiments.
    4. Suggestions or insights for potential investors or stakeholders.

    Be concise but ensure that the report is actionable and insightful.
    """
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert in financial sentiment analysis."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500
    )
    return response.choices[0].message.content.strip()

def query_and_generate_reports(queries: List[str]) -> List[Dict[str, str]]:
    """
    Query the knowledge graph for each query, aggregate context, and generate summary reports.
    Returns a list of dictionaries containing the query, aggregated context, and generated report.
    """
    results = []

    for query in queries:
        print(f"Processing query: {query}")
        context = query_engine.query(query)

        # Generate a summary report using the aggregated context
        report = generate_summary_report(context, query)

        results.append({
            "query": query,
            "context": context,
            "report": report
        })

    return results

def save_reports_to_file(results: List[Dict[str, str]], filename: str):
    """
    Save query results and their generated reports to a file.
    """
    with open(filename, "w", encoding="utf-8") as file:
        for result in results:
            file.write(f"Query:\n{result['query']}\n\n")
            file.write(f"Context:\n{result['context']}\n\n")
            file.write(f"Generated Report:\n{result['report']}\n\n")
            file.write("-" * 80 + "\n\n")



# Create the query engine
query_engine = index.as_query_engine(include_text=True)

# Define a list of queries, Different kinds of queries to see the effectiveness of in EV sector
queries = [
    "How to invest in the EV sector? Summarize the most important financial trends in the EV Sector.",
    "What are the recent financial sentiments about renewable energy investments?",
    "Summarize the financial outlook for the technology sector in 2024.",
    "What are the key financial risks in the automotive industry this year?",
    "Provide insights on the financial performance of AI startups in the US."
]

# Execute the queries and generate reports
results = query_and_generate_reports(queries)

# Save the reports to a file
output_file = "financial_sentiment_reports.txt"
save_reports_to_file(results, output_file)

# Print a summary of the generated reports
for result in results:
    print(f"Query: {result['query']}")
    print(f"Generated Report:\n{result['report']}")
    print("-" * 80)


Processing query: How to invest in the EV sector? Summarize the most important financial trends in the EV Sector.
Processing query: What are the recent financial sentiments about renewable energy investments?


/usr/local/lib/python3.11/dist-packages/neo4j/_codec/hydration/v1/hydration_handler.py:61: RuntimeWarning: coroutine 'PGRetriever._aretrieve' was never awaited
  b"R": self.hydrate_relationship,


Processing query: Summarize the financial outlook for the technology sector in 2024.
Processing query: What are the key financial risks in the automotive industry this year?
Processing query: Provide insights on the financial performance of AI startups in the US.
Query: How to invest in the EV sector? Summarize the most important financial trends in the EV Sector.
Generated Report:
Report:

Title: Financial Trends and Sentiment Analysis in the EV Sector

1. High-Level Summary:
The electric vehicle (EV) sector's financial landscape is experiencing an upward trend due to increasing revenues ensuing from the higher sales and growing popularity of EVs. With steady growth in regulatory credits and the expansion of manufacturers into other service areas such as energy storage deployment and charging networks, the sector is witnessing dynamic revenue growth and improved financial stability.

2. Sentiment Analysis:

   a) Positive Sentiment: There is a strong positive sentiment around the incr

### **Conclusion**

This project successfully demonstrates the power of combining graph-based knowledge representation with Retrieval-Augmented Generation (RAG) to enable effective financial sentiment analysis. By leveraging cutting-edge technologies such as LlamaIndex, Neo4j, and OpenAI embeddings, the system efficiently organizes and retrieves data from unstructured sources, transforming it into actionable insights. The integration of knowledge graphs allows for a rich understanding of relationships between entities, while the use of LLMs enhances contextual accuracy in query responses.